In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

## 1.车牌提取


In [33]:
root = 'hw2/hw2-task1'
    
# 读取原始图片
img = cv2.imread('hw2/license+plate.jpg')
new_img = img.copy()
def torgb(src):
    return cv2.cvtColor(src, cv2.COLOR_HSV2RGB)

# 定义mark
# lower_blue = np.array([90, 90, 0])
# upper_blue = np.array([120, 210, 250])
lower_blue = np.array([100, 60, 60])
upper_blue = np.array([120, 210, 250])
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mark = cv2.inRange(hsv, lower_blue, upper_blue)
cv2.imwrite(f"{root}/mark.jpg", mark)

# 形态学操作
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
img_erode = cv2.erode(mark, kernel, iterations=1)
img_dilated = cv2.dilate(mark, kernel, iterations=3)
cv2.imwrite(f"{root}/dilated.jpg", img_dilated)

# findCounters: 寻找轮廓
contours, hierarchy = cv2.findContours(img_dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

# 找寻车牌面积均值
avg_area = 0
for value in contours:
    avg_area += cv2.contourArea(value)
avg_area /= len(contours)

# 提取矩形坐标
rectangles = []
image_list = []
for value in contours:
    area = cv2.contourArea(value)
    # 如果contour的面积大于阈值
    if area > 2*avg_area:
        rect = cv2.minAreaRect(value)
        # box = cv2.boxPoints(rect)
        # box = np.int0(box)  
        # angle =abs(abs(rect[2])-45) 
        # length = max(rect[1])
        # wideth = min(rect[1])
        area = rect[1][0] * rect[1][1]
        x = []
        y = []
        for point in value:
            y.append(point[0][0])
            x.append(point[0][1])
        r = [min(y), min(x), max(y), max(x)]
        rectangles.append(r)
        if abs(rect[2]) < 45:
            a = rect[0][0]-0.5*rect[1][0]
            b = rect[0][0]+0.5*rect[1][0]
            c = rect[0][1]-0.5*rect[1][1]
            d = rect[0][1]+0.5*rect[1][1]
        else:
            a = rect[0][0]-0.5*rect[1][1]
            b = rect[0][0]+0.5*rect[1][1]
            c = rect[0][1]-0.5*rect[1][0]
            d = rect[0][1]+0.5*rect[1][0]
        # 提取识别到的车牌图片
        license_image = new_img[round(c):round(d), round(a):round(b)]
        image_list.append(license_image)
        # 画出识别到的车标轮廓
        cv2.rectangle(img, (r[0], r[1]), (r[2], r[3]), (0, 255, 0), 2)
        cv2.imwrite(f'{root}/license.jpg', img)

# 对识别到的车牌进行字符提取
for j in range(len(image_list)):
    # 保存图片
    path = f'hw2/hw2-task1/figure-{j}/'
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(f'{path}license.jpg', image_list[j])

    # 预处理
    img_gray = cv2.cvtColor(image_list[j], cv2.COLOR_BGR2GRAY)
    ret, img_binary = cv2.threshold(img_gray, 110, 255, cv2.THRESH_BINARY)
    cv2.imwrite(f'{path}binary.jpg', img_binary)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    # img_binary = cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, kernel)
    # img_binary = cv2.morphologyEx(img_binary, cv2.MORPH_OPEN, kernel)
    # cv2.imshow('open', img_binary)
    img_bianry = cv2.erode(img_binary, kernel, iterations=3)
    cv2.imwrite(f"{path}dilated.jpg", img_binary)

    # 轮廓提取
    contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 寻找车牌中的contour面积均值
    area_avg = 0
    height_avg = 0
    for i in range(len(contours)):
        x, y, w, h = cv2.boundingRect(contours[i])
        area_avg += w*h
        height_avg += h
    area_avg /= len(contours)
    height_avg /= len(contours)

    # 识别车牌汇总的字符
    location_list = []
    count = 0
    for value in contours:
        x, y, w, h = cv2.boundingRect(value)
        area = w*h
        # ratio>1: too tall, ratio<1: too wide
        ratio =  h/w
        if area < area_avg/10 or area > area_avg*3 or ratio < 1 or h < 2*height_avg/3:
            continue
        location_list.append((x,y,w,h))
        count += 1
    
    def takefirst(elem):
        return elem[0]
    location_list.sort(key=takefirst)

    for i in range(0, count):
        char_image  = image_list[j][ location_list[i][1]:location_list[i][1] + location_list[i][3], location_list[i][0] : location_list[i][0]+location_list[i][2]] 
        file_name = 'img' + str(i) + '.jpg'
        cv2.imwrite(path + ('img' + str(i) + '.jpg'), char_image)
        cv2.rectangle(image_list[j], (location_list[i][0], location_list[i][1]-5), (location_list[i][0] + location_list[i][2], location_list[i][1] + location_list[i][3]), (0, 255, 0), 1)
        cv2.imwrite(path+'letter.jpg', image_list[j])
cv2.waitKey(0)
cv2.destroyAllWindows()

## 2.文字区域识别

In [2]:
"""
首先找答题标号区域，然后根据答题标号区域进行划分
"""
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from swtloc import SWTLocalizer
from swtloc.utils import imgshowN, imgshow

root = 'hw2/hw2-task2'
img = cv2.imread('hw2/text+block.jpg')

# 二值化
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, img_binary = cv2.threshold(img_gray, 200, 255, cv2.THRESH_BINARY)
cv2.imshow('binary', img_binary)

# 形态学操作
img_edge = img_binary.copy()
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
img_edge = cv2.erode(img_edge, kernel, iterations=1)
cv2.imshow('dilated', img_edge)

contours, hierarchy = cv2.findContours(img_edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for value in contours:
    x, y, w, h = cv2.boundingRect(value)
    cv2.rectangle(img_binary, (x,y), (x+w, y+h), (0,255,0), 1)

cv2.imshow('res', img_binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3.答题卡信息提取

In [1]:
"""
首先找答题标号区域，然后根据答题标号区域进行划分
"""
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

root = 'hw2/hw2-task3'
img = cv2.imread('hw2/cards.jpeg')

# 二值化
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, img_binary = cv2.threshold(img_gray, 125, 255, cv2.THRESH_BINARY)
cv2.imshow('binary', img_binary)
cv2.imwrite(f'{root}/binary.jpg', img_binary)

# 边缘提取
img_edge = cv2.Canny(img_binary, 30, 120)
cv2.imshow('edge', img_edge)
cv2.imwrite(f'{root}/edge.jpg', img_edge)

# 形态学操作
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
img_edge = cv2.dilate(img_edge, kernel, iterations=4)
img_edge = cv2.erode(img_edge, kernel, iterations=2)
cv2.imshow('dilated', img_edge)
cv2.imwrite(f'{root}/dilated.jpg', img_edge)

contours, hierarchy = cv2.findContours(img_edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

max_area = 0
answer = []
question = []
question_x = []
answer_x = 0
width_avg = 0
height_avg = 0
count = 0
count_question = 0
for value in contours:
    x, y, w, h = cv2.boundingRect(value)
    if w*h > max_area:
        max_area = w*h
for value in contours:
    x, y, w, h = cv2.boundingRect(value)
    if w*h > max_area/2:
        width_avg += w
        height_avg += h
        count_question += 1
        for i in range(5):
            space_y = int(h/5)
            question.append([x, y+space_y*i, x+space_y, y+space_y*(i+1)])
            question_x.append(x)
    else:
        answer_x += w
        count += 1 

answer_x = int(answer_x/count)
width_avg = int(width_avg/count_question)
height_avg = int(height_avg/(count_question*5))
question = sorted(question)

# 获取答题区域分割线坐标
question_x = list(set(question_x))
question_x = sorted(question_x)
question_x.append(img.shape[1])
count = 0
while question_x[count] != question_x[-1]:
    if question_x[count+1] - question_x[count] < 10:
        del question_x[count+1]
    else:
        count +=1

# 对标号区域进行对齐
for value in question:
    for item in question_x:
        if abs(item-value[0])<10:
            value[0] = item
            break
    value[2] = value[0]+width_avg
    value[3] = value[1]+height_avg
    cv2.rectangle(img, (value[0], value[1]), (value[2], value[3]), (0, 255, 0), 1)
cv2.imwrite(f'{root}/question.jpg', img)
# 划分答题区域
question = sorted(question)
count = 0
for value in question:
    count += 1
    index = question_x.index(value[0])
    circle = 1
    space_x = 10
    while True:
        add_x = value[2] + (answer_x+space_x)*circle
        if add_x < question_x[index+1]-20:
            new_x = value[2] + space_x + (answer_x+13)*(circle-1)
            cv2.rectangle(img, (new_x, value[1]), (new_x+answer_x, value[3]), (0, 0, 255), 1)
            answer.append([new_x, value[1], new_x+answer_x, value[3], count, circle])
            circle += 1
        else:
            break
cv2.imwrite(f'{root}/answer.jpg', img)
answer = sorted(answer)

# 判断答题区域是否有答案
result_dict={}
for i in range(len(question)):
    result_dict[i+1] = []
for i in range(len(answer)):
    value = answer[i]
    answer_block = img_binary[value[1]:value[3], value[0]:value[2]]
    count_black = len(answer_block[answer_block==0])
    count_white = len(answer_block[answer_block==255])
    if count_black/(count_black+count_white) >= 0.2:
        cv2.rectangle(img, (value[0], value[1]), (value[2], value[3]), (255, 0, 0), 1)
        result_dict[value[4]].append(chr(value[5]+64))

print(result_dict)
cv2.imshow('rectangle', img)
cv2.imwrite(f'{root}/result.jpg', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{1: [], 2: ['B'], 3: ['C'], 4: [], 5: ['D'], 6: [], 7: ['A'], 8: ['D'], 9: [], 10: ['B'], 11: ['A'], 12: ['A'], 13: ['B'], 14: ['B'], 15: [], 16: ['B'], 17: [], 18: ['A'], 19: ['C'], 20: [], 21: [], 22: ['C'], 23: [], 24: ['A'], 25: [], 26: [], 27: ['B'], 28: [], 29: ['B'], 30: [], 31: ['C'], 32: [], 33: ['B'], 34: ['C'], 35: [], 36: [], 37: [], 38: ['B'], 39: [], 40: [], 41: [], 42: [], 43: ['A', 'B'], 44: [], 45: []}
